In [1]:
import cymetric as cym
import os
import pandas as pd
from pandas.util.testing import assert_frame_equal
import numpy as np
from uuid import UUID
from cymetric.tools import raw_to_series, ensure_dt_bytes




/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
# Open the database
db = cym.dbopen("test-input.h5")
metadata = cym.root_metric(name='Metadata')

In [3]:
# Define an Evaluator, allows to acces the different data tables in the database
myEval = cym.Evaluator(db)

In [4]:
# Acces on specific table (here Materials)
metadata = myEval.eval('Metadata')
time = myEval.eval('TimeList')
agent = myEval.eval('AgentEntry')

In [5]:
deployment = metadata[ metadata['Type'] == "deployment" ]
deployment_agent = agent[agent['AgentId'].isin(deployment['AgentId'])]

In [7]:
deployment_agent

,SimId,AgentId,Kind,Spec,Prototype,ParentId,Lifetime,EnterTime
5,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,Facility,:cycamore:Reactor,Reactor2,12,-1,2
6,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,17,Facility,:cycamore:Reactor,Reactor3,12,-1,3


In [8]:
metadata

,SimId,AgentId,keyword,Type,Value
0,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,decommission,25.000000
1,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,deployment,45.000000
2,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,timestep,35.000000
3,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,throughput,15.000000
4,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,land,decommission,24.000000
5,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,lolipop,throughput,11.000000
6,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,manpower,timestep,32.000000
7,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,water,deployment,43.000000
8,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,17,co2,decommission,25.000000
9,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,17,co2,deployment,45.000000


In [7]:
_tmp = pd.merge(metadata[['SimId', 'AgentId','keyword', 'Value']], deployment_agent, on=['SimId', 'AgentId'])

In [9]:
deployment_use = pd.DataFrame(data={'SimId': _tmp.SimId, 'AgentId': _tmp.AgentId, 'Time': _tmp.EnterTime, 'keyword': _tmp.keyword, 'Value':_tmp.Value}, 
                              columns=['SimId', 'AgentId', 'keyword', 'Value'])

In [10]:
deployment_use

,SimId,AgentId,keyword,Value
0,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,25.000000
1,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,45.000000
2,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,35.000000
3,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,co2,15.000000
4,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,land,24.000000
5,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,lolipop,11.000000
6,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,manpower,32.000000
7,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,16,water,43.000000
8,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,17,co2,25.000000
9,f1538a29-98e8-4d7e-81a4-20f4a3c8c946,17,co2,45.000000
